In [157]:
import srd
from srd import federal, quebec, tax, Person, Hhold, Dependent
from srd.federal import covidfed
from srd.quebec import covidqc

In [158]:
jean = srd.Person(age=45,earn=50e3)
pauline = srd.Person(age=45,earn=25e3)

In [159]:
hh = srd.Hhold(jean,pauline,prov='qc')

In [160]:
fed_form = federal.form(2020, covidfed.policyfed(icerb=False))
pro_form = quebec.form(2020, covidqc.policyqc(iiprew=False))
tax_form = srd.tax(2020)
tax_form.compute(hh)
vars(tax_form)

{'year': 2020,
 'ifed': True,
 'iprov': True,
 'ipayroll': True,
 'ioas': True,
 'iass': True,
 'policyqc': <srd.quebec.covidqc.policyqc at 0x1bed1310fd0>,
 'policyfed': <srd.federal.covidfed.policyfed at 0x1bed2f1ed30>,
 'payroll': <srd.payroll.payroll at 0x1beee2cffd0>,
 'covidqc': <srd.quebec.covidqc.programs at 0x1beee3603a0>,
 'covidfed': <srd.federal.covidfed.programs at 0x1beee360e80>,
 'federal': <srd.federal.forms.form_2020 at 0x1beee3609a0>,
 'prov': {'qc': <srd.quebec.forms.form_2020 at 0x1beee3608b0>,
  'on': <srd.ontario.forms.form_2020 at 0x1beee360670>},
 'oas': <srd.oas.programs.program_2020 at 0x1beee360c10>,
 'ass': <srd.assistance.programs.program_2020 at 0x1beee360e50>}

In [161]:
vars(fed_form.policyfed)


{'icerb': False,
 'icesb': True,
 'icovid_gst': True,
 'icovid_ccb': True,
 'iei': False}

In [162]:
vars(pro_form.policyqc)


{'iiprew': False}

In [163]:
vars(tax_form.covidfed)


{'begin_april': 3,
 'cerb_base': 2000.0,
 'cerb_min_inc_work': 5000.0,
 'cerb_max_earn': 1000.0,
 'cesb_base': 1250.0,
 'cesb_supp': 750.0,
 'cesb_max_earn': 1000.0,
 'policy': <srd.federal.covidfed.policyfed at 0x1bed2f1ed30>}

In [164]:
vars(tax_form.covidqc)


{'begin_april': 3,
 'iprew_min_inc_work': 5000.0,
 'iprew_max_inc_tot': 28600.0,
 'iprew_monthly_amount': 400.0,
 'iprew_max_earn': 2500.0,
 'iprew_max_months': 4,
 'policy': <srd.quebec.covidqc.policyqc at 0x1bed1310fd0>}

In [165]:
## test CERB
tax_form = tax(2020)
p = Person(earn=6000, months_cerb_cesb=2)
hh = Hhold(p, prov='qc')
tax_form.compute(hh)
print(p.inc_cerb==4000)


True


In [166]:
## test no CERB
tax_form = srd.tax(2020)
p = srd.Person(earn=[2000]*12, months_cerb_cesb=2)
hh = srd.Hhold(p, prov='qc')
tax_form.compute(hh)
print(p.inc_cerb==0)

True


In [167]:
## test CESB
tax_form = srd.tax(2020)
p = srd.Person(earn=6000, months_cerb_cesb=2, student=True)
hh = srd.Hhold(p, prov='qc')
tax_form.compute(hh)
print(p.inc_cerb==0)
print(p.inc_cesb==2500)

True
True


In [168]:
## test disabled CESB
tax_form = tax(2020)
p = Person(earn=6000, months_cerb_cesb=2, student=True, disabled=True)
hh = Hhold(p, prov='qc')
tax_form.compute(hh)
print(p.inc_cerb==0)
print(p.inc_cesb==4000)


True
True


In [169]:
## test no cesb student
tax_form = tax(2020)
p = Person(earn=13000, months_cerb_cesb=2, student=True)
hh = Hhold(p, prov='qc')
tax_form.compute(hh)
print(p.inc_cesb==0)

True


In [170]:
## test supp_dep_ces
tax_form = tax(2020)
p = Person(months_cerb_cesb=1, student=True)
hh = Hhold(p, prov='qc')
dep = Dependent(age=12)
hh.dep.append(dep)
tax_form.compute(hh)
print(p.inc_cesb==2000)

True


In [171]:
## test couple cesb diabled
tax_form = tax(2020)
p0 = Person(months_cerb_cesb=1, student=True)
p1 = Person(months_cerb_cesb=3, student=True, disabled=True)
hh = Hhold(p0, p1, prov='qc')
tax_form.compute(hh)
print(p0.inc_cesb==1250)
print(p1.inc_cesb==6000)

True
True


In [172]:
## test couple cesb dep
tax_form = tax(2020)
p0 = Person(months_cerb_cesb=1, student=True)
p1 = Person(months_cerb_cesb=2, student=True)
hh = Hhold(p0, p1, prov='qc')
dep = Dependent(age=12)
hh.dep.append(dep)
tax_form.compute(hh)
print(p0.inc_cesb==1625)
print(p1.inc_cesb==3250)

True
True


In [173]:
## test couple cesb dep disable
tax_form = tax(2020)
p0 = Person(months_cerb_cesb=1, student=True, disabled=True)
p1 = Person(months_cerb_cesb=2, student=True)
hh = Hhold(p0, p1, prov='qc')
dep = Dependent(age=12)
hh.dep.append(dep)
tax_form.compute(hh)
print(p0.inc_cesb==2000)
print(p1.inc_cesb==4000)

True
True


In [174]:
## test iprew
tax_form = tax(2020)
p = Person(essential_worker=True, earn=6000)
hh = Hhold(p, prov='qc')
tax_form.compute(hh)
print(p.inc_cesb==0)
print(p.covid['iprew']==1600)

True
True


In [175]:
## test no iprew
tax_form = tax(2020)

p = Person(essential_worker=False, earn=6000)
hh = Hhold(p, prov='qc')
tax_form.compute(hh)
print(p.covid['iprew']==0)

p = Person(essential_worker=True, earn=4000)
hh = Hhold(p, prov='qc')
tax_form.compute(hh)
print(p.covid['iprew']==0)

p = Person(essential_worker=True, earn=6000, othntax=30000)
hh = Hhold(p, prov='qc')
tax_form.compute(hh)
print(p.covid['iprew']==0)

p = Person(essential_worker=True, earn=29000)
hh = Hhold(p, prov='qc')
tax_form.compute(hh)
print(p.covid['iprew']==0)

True
True
True
True


In [176]:
# test iprew 3 months
tax_form = tax(2020)
p = Person(essential_worker=True, earn=[2000]*6 + [2600]*6)
hh = Hhold(p, prov='qc')
tax_form.compute(hh)
print(p.covid['iprew']==1200)


True
